In [1]:
# train.py (esqueleto)
import pandas as pd
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib
import numpy as np

# 1) Cargar datos
df = pd.read_csv("data/public_dataset.csv")
y = df["label"]
X = df.drop(columns=["label"])

# 2) Definir columnas
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

# 3) Pipeline de preprocesamiento
preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                          ("sc", StandardScaler())]), num_cols),
        ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                          ("oh", OneHotEncoder(handle_unknown="ignore"))]), cat_cols),
    ]
)

X_proc = preprocess.fit_transform(X)
joblib.dump(preprocess, "models/preprocess.joblib")

# 4) Tensores
X_train, X_val, y_train, y_val = train_test_split(X_proc, y, test_size=0.2, stratify=y, random_state=42)
X_train_t = torch.tensor(X_train.toarray() if hasattr(X_train, "toarray") else X_train, dtype=torch.float32)
X_val_t   = torch.tensor(X_val.toarray()   if hasattr(X_val,   "toarray")   else X_val,   dtype=torch.float32)
y_train_t = torch.tensor(y_train.values, dtype=torch.long)
y_val_t   = torch.tensor(y_val.values,   dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=128, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val_t,   y_val_t),   batch_size=256)

# 5) Modelo
input_dim = X_train_t.shape[1]
n_classes = len(np.unique(y))
model = nn.Sequential(
    nn.Linear(input_dim, 256), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.2),
    nn.Linear(128, n_classes)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
crit = nn.CrossEntropyLoss()

# 6) Entrenar (early stopping simple)
best_val = float("inf")
patience, wait = 10, 0
for epoch in range(200):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        loss = crit(model(xb), yb)
        loss.backward(); opt.step()
    # validar
    model.eval()
    with torch.no_grad():
        val_losses = []
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            val_losses.append(crit(model(xb), yb).item())
    val_loss = np.mean(val_losses)
    if val_loss < best_val:
        best_val = val_loss; wait = 0
        torch.save(model.state_dict(), "models/model_state.pth")
    else:
        wait += 1
        if wait >= patience: break


ModuleNotFoundError: No module named 'torch'